In [1]:
import pickle
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score
import matplotlib.pylab as plt
#from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict, ShuffleSplit
import tsfresh as tf
%matplotlib inline


C:\Users\AOX\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [24]:
df=pd.read_csv('data.csv',header=None)
dfc=df.values

In [31]:
df.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,3,0,0,1,1,654.0,0.65,0.64,-1.12,-0.05,-52.19,74.92,-254.36
1,3,0,0,1,2,892.0,0.66,0.57,-0.77,0.22,-16.69,10.38,-281.18
2,3,0,0,1,3,281.0,0.63,0.42,-0.92,0.11,-46.56,93.08,-280.71
3,3,0,0,1,4,358.0,0.61,0.24,0.84,-0.99,317.01,18.46,236.56
4,3,0,0,1,5,472.0,0.65,-0.48,0.23,-0.69,279.54,103.47,-18.27
5,3,0,0,1,6,7.0,0.33,-0.57,0.03,-0.19,-14.57,-107.82,241.41
6,3,1,0,1,1,664.0,0.98,0.03,-1.84,0.12,-48.55,97.91,-403.20
7,3,1,0,1,2,893.0,1.00,-0.07,-1.25,0.32,1.60,5.92,-437.87
8,3,1,0,1,3,269.0,0.93,-0.44,-1.22,0.29,-55.90,108.08,-440.27
9,3,1,0,1,4,429.0,0.86,-0.20,1.49,-0.85,472.85,36.07,293.73


In [154]:
for i in range(0,len(dfc)):
    dfc[i][10]=dfc[i][10]*math.pi/180
    dfc[i][11]=dfc[i][11]*math.pi/180
    dfc[i][12]=dfc[i][12]*math.pi/180
dfc

array([[  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.77472150e-04,   3.98317944e-04,  -1.35232451e-03],
       [  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -8.87336690e-05,   5.51860686e-05,  -1.49491510e-03],
       [  3.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.47539822e-04,   4.94866981e-04,  -1.49241631e-03],
       ..., 
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
          3.93426693e-06,  -4.23093192e-04,   2.89221785e-04],
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
         -4.32290871e-04,   3.32817716e-04,  -1.86558685e-04],
       [  2.99000000e+02,   5.90000000e+01,   0.00000000e+00, ...,
         -5.75732116e-04,   3.11976734e-04,  -2.09845292e-04]])

In [194]:
cdf=np.zeros((17820,47))
fdf=np.zeros((17820,47))
rdf=np.zeros((17820,47))
classes=np.zeros(297)
fclasses=np.zeros(297)
rclasses=np.zeros(297)
for i in range(17820):
    for k in range(5):
        for m in range(8):
            df[i][k*8+m]=dfc[i*6+k][5+m]
            rdf[i][k*8+m]=dfc[i*6+k][5+m]
            fdf[i][k*8+m]=dfc[i*6+k][5+m]
    for m in range(7):
        cdf[i][40+m]=dfc[i*6+5][6+m]
        fdf[i][40+m]=dfc[i*6+5][6+m]
        rdf[i][40+m]=dfc[i*6+5][6+m]
    if(i%60==0):
        n=i/60
        classes[int(n)]=dfc[i*6][2]*8+dfc[i*6][3]
        fclasses[int(n)]=dfc[i*6][2]
        rclasses[int(n)]=dfc[i*6][3]

In [195]:
counts=np.zeros(297)
fcounts=np.zeros(6)
rcounts=np.zeros(8)
for i in range(len(rclasses)):
    counts[int(classes[i])]+=1
    fcounts[int(fclasses[i])]+=1
    rcounts[int(rclasses[i])]+=1
rcounts

array([ 37.,  44.,  34.,  34.,  43.,  37.,  24.,  44.])

In [196]:
cdf

array([[  6.54000000e+02,   6.50000000e-01,   6.40000000e-01, ...,
         -7.74625259e-05,  -5.73233325e-04,   1.28347484e-03],
       [  6.64000000e+02,   9.80000000e-01,   3.00000000e-02, ...,
         -1.26215536e-04,  -7.49903177e-04,   1.97372602e-03],
       [  6.64000000e+02,   9.90000000e-01,  -1.29000000e+00, ...,
         -2.38129481e-04,  -5.77805581e-04,   2.06416099e-03],
       ..., 
       [  6.52000000e+02,   8.10000000e-01,  -3.30000000e-01, ...,
         -1.04630234e-04,  -6.05770776e-04,   3.98743270e-06],
       [  6.44000000e+02,   8.40000000e-01,  -2.30000000e-01, ...,
         -3.84229015e-04,  -1.46631192e-04,  -6.68293721e-05],
       [  6.34000000e+02,   8.50000000e-01,  -4.20000000e-01, ...,
         -5.75732116e-04,   3.11976734e-04,  -2.09845292e-04]])

In [200]:
cdf=pd.DataFrame(cdf)
fdf=pd.DataFrame(fdf)
rdf=pd.DataFrame(rdf)
cdf.head(5)

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,654.0,0.65,0.64,-1.12,-0.05,-0.000277,0.000398,-0.001352,892.0,0.66,...,0.001486,0.000550,-0.000097,0.33,-0.57,0.03,-0.19,-0.000077,-0.000573,0.001283
1,664.0,0.98,0.03,-1.84,0.12,-0.000258,0.000521,-0.002144,893.0,1.00,...,0.002132,0.000851,-0.000306,0.67,-0.67,-0.40,-0.41,-0.000126,-0.000750,0.001974
2,664.0,0.99,-1.29,-2.10,0.31,0.000085,0.000319,-0.002325,888.0,1.00,...,0.001961,0.000956,-0.000651,1.00,-0.51,-1.22,-0.62,-0.000238,-0.000578,0.002064
3,665.0,1.00,-1.77,-2.03,0.42,0.000458,-0.000092,-0.002197,892.0,1.00,...,0.001438,0.001191,-0.001043,1.00,-0.39,-1.44,-0.53,-0.000753,-0.000216,0.001948
4,634.0,0.98,-1.65,-1.70,0.04,0.000615,-0.000329,-0.001920,263.0,0.99,...,0.001039,0.001345,-0.001158,1.00,-0.46,-1.69,-0.41,-0.001054,-0.000032,0.001705


In [207]:
fdf
fdfd=pd.DataFrame(np.zeros((297,967)))
for i in range(297):
    for coli in range(46):
        for k in range(0,7):
            fdfd.iloc[i][(coli*7)+k] = fdf.iloc[(i*60):(i*60)+30][coli].describe()[[k+1]]
            fdfd.iloc[i][(coli*7)+k+322] = fdf.iloc[(i*60)+30:(i*60)+60][coli].describe()[[k+1]]
            fdfd.iloc[i][(coli*7)+k+644] = fdf.iloc[(i*60):(i*60)+60][coli].describe()[[k+1]]
    fdfd.iloc[i][966]=fclasses[i]
    print(i)
fdfd

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,0,1,2,3,4,5,6,7,8,9,...,957,958,959,960,961,962,963,964,965,966
0,628.833333,92.826528,310.0,638.00,654.5,668.75,711.0,0.920667,0.115518,0.52,...,0.000321,0.000557,-8.221909e-05,0.000267,-0.000750,-0.000248,-0.000099,0.000121,0.000393,0.0
1,542.166667,76.490289,412.0,466.75,552.0,618.50,655.0,0.763000,0.104853,0.61,...,0.000286,0.001931,2.450455e-04,0.000937,-0.001562,-0.000343,0.000098,0.000673,0.002812,0.0
2,675.733333,22.400636,646.0,663.50,669.0,682.00,744.0,0.946000,0.017140,0.92,...,0.000482,0.001575,-3.131366e-04,0.000661,-0.001519,-0.000692,-0.000195,0.000147,0.000832,1.0
3,416.633333,204.512622,80.0,227.00,460.5,621.00,672.0,0.585000,0.267101,0.19,...,0.000336,0.001177,-4.202125e-04,0.001085,-0.003657,-0.000412,-0.000106,0.000162,0.001066,5.0
4,300.733333,125.416804,21.0,234.75,331.0,391.75,487.0,0.415333,0.145169,0.10,...,0.000047,0.001362,-1.552706e-05,0.001403,-0.003996,-0.000002,0.000121,0.000459,0.002704,4.0
5,233.633333,99.883202,59.0,171.50,223.0,331.50,395.0,0.333333,0.113147,0.17,...,0.000095,0.001096,-1.213526e-04,0.001297,-0.004021,0.000064,0.000135,0.000318,0.002263,4.0
6,268.466667,100.490773,46.0,194.50,290.5,352.00,412.0,0.353333,0.121523,0.14,...,0.000034,0.000789,2.589979e-04,0.001348,-0.003887,-0.000019,0.000177,0.000607,0.003511,3.0
7,471.633333,156.561211,195.0,374.00,439.5,637.75,699.0,0.639333,0.209102,0.32,...,0.000159,0.001333,-1.017690e-04,0.001426,-0.004315,-0.000203,0.000156,0.000697,0.002354,3.0
8,491.500000,101.732831,296.0,402.25,518.5,583.75,624.0,0.663667,0.130212,0.42,...,0.000182,0.001340,1.235732e-04,0.001335,-0.003162,-0.000149,0.000335,0.000771,0.002529,1.0
9,632.933333,23.431387,583.0,613.50,632.0,653.00,676.0,0.851000,0.027959,0.80,...,0.000164,0.001250,3.424638e-04,0.000472,-0.000332,0.000006,0.000185,0.000561,0.001439,1.0


In [212]:
fdfd
fdfd.to_csv('trainingset.csv')

In [219]:
train=fdfd.iloc[0:297,0:966]
train

,0,1,2,3,4,5,6,7,8,9,...,956,957,958,959,960,961,962,963,964,965
0,628.833333,92.826528,310.0,638.00,654.5,668.75,711.0,0.920667,0.115518,0.52,...,0.000054,0.000321,0.000557,-8.221909e-05,0.000267,-0.000750,-0.000248,-0.000099,0.000121,0.000393
1,542.166667,76.490289,412.0,466.75,552.0,618.50,655.0,0.763000,0.104853,0.61,...,-0.000291,0.000286,0.001931,2.450455e-04,0.000937,-0.001562,-0.000343,0.000098,0.000673,0.002812
2,675.733333,22.400636,646.0,663.50,669.0,682.00,744.0,0.946000,0.017140,0.92,...,-0.000047,0.000482,0.001575,-3.131366e-04,0.000661,-0.001519,-0.000692,-0.000195,0.000147,0.000832
3,416.633333,204.512622,80.0,227.00,460.5,621.00,672.0,0.585000,0.267101,0.19,...,0.000064,0.000336,0.001177,-4.202125e-04,0.001085,-0.003657,-0.000412,-0.000106,0.000162,0.001066
4,300.733333,125.416804,21.0,234.75,331.0,391.75,487.0,0.415333,0.145169,0.10,...,-0.000091,0.000047,0.001362,-1.552706e-05,0.001403,-0.003996,-0.000002,0.000121,0.000459,0.002704
5,233.633333,99.883202,59.0,171.50,223.0,331.50,395.0,0.333333,0.113147,0.17,...,-0.000052,0.000095,0.001096,-1.213526e-04,0.001297,-0.004021,0.000064,0.000135,0.000318,0.002263
6,268.466667,100.490773,46.0,194.50,290.5,352.00,412.0,0.353333,0.121523,0.14,...,-0.000153,0.000034,0.000789,2.589979e-04,0.001348,-0.003887,-0.000019,0.000177,0.000607,0.003511
7,471.633333,156.561211,195.0,374.00,439.5,637.75,699.0,0.639333,0.209102,0.32,...,0.000055,0.000159,0.001333,-1.017690e-04,0.001426,-0.004315,-0.000203,0.000156,0.000697,0.002354
8,491.500000,101.732831,296.0,402.25,518.5,583.75,624.0,0.663667,0.130212,0.42,...,0.000065,0.000182,0.001340,1.235732e-04,0.001335,-0.003162,-0.000149,0.000335,0.000771,0.002529
9,632.933333,23.431387,583.0,613.50,632.0,653.00,676.0,0.851000,0.027959,0.80,...,-0.000228,0.000164,0.001250,3.424638e-04,0.000472,-0.000332,0.000006,0.000185,0.000561,0.001439


In [258]:
classpd=pd.DataFrame(classes)
fclasspd=pd.DataFrame(fclasses)
rclasspd=pd.DataFrame(rclasses)
cdft=pd.concat([train,classpd],axis=1)
fdft=pd.concat([train,fclasspd],axis=1)
rdft=pd.concat([train,rclasspd],axis=1)
cdft=rdft.rename(columns={ cdft.columns[966]: "966" })
fdft=fdft.rename(columns={ fdft.columns[966]: "966" })
rdft=rdft.rename(columns={ rdft.columns[966]: "966" })
fdft

,966,1,2,3,4,5,6,7,8,9,...,957,958,959,960,961,962,963,964,965,966
0,628.833333,92.826528,310.0,638.00,654.5,668.75,711.0,0.920667,0.115518,0.52,...,0.000321,0.000557,-8.221909e-05,0.000267,-0.000750,-0.000248,-0.000099,0.000121,0.000393,0.0
1,542.166667,76.490289,412.0,466.75,552.0,618.50,655.0,0.763000,0.104853,0.61,...,0.000286,0.001931,2.450455e-04,0.000937,-0.001562,-0.000343,0.000098,0.000673,0.002812,0.0
2,675.733333,22.400636,646.0,663.50,669.0,682.00,744.0,0.946000,0.017140,0.92,...,0.000482,0.001575,-3.131366e-04,0.000661,-0.001519,-0.000692,-0.000195,0.000147,0.000832,1.0
3,416.633333,204.512622,80.0,227.00,460.5,621.00,672.0,0.585000,0.267101,0.19,...,0.000336,0.001177,-4.202125e-04,0.001085,-0.003657,-0.000412,-0.000106,0.000162,0.001066,5.0
4,300.733333,125.416804,21.0,234.75,331.0,391.75,487.0,0.415333,0.145169,0.10,...,0.000047,0.001362,-1.552706e-05,0.001403,-0.003996,-0.000002,0.000121,0.000459,0.002704,4.0
5,233.633333,99.883202,59.0,171.50,223.0,331.50,395.0,0.333333,0.113147,0.17,...,0.000095,0.001096,-1.213526e-04,0.001297,-0.004021,0.000064,0.000135,0.000318,0.002263,4.0
6,268.466667,100.490773,46.0,194.50,290.5,352.00,412.0,0.353333,0.121523,0.14,...,0.000034,0.000789,2.589979e-04,0.001348,-0.003887,-0.000019,0.000177,0.000607,0.003511,3.0
7,471.633333,156.561211,195.0,374.00,439.5,637.75,699.0,0.639333,0.209102,0.32,...,0.000159,0.001333,-1.017690e-04,0.001426,-0.004315,-0.000203,0.000156,0.000697,0.002354,3.0
8,491.500000,101.732831,296.0,402.25,518.5,583.75,624.0,0.663667,0.130212,0.42,...,0.000182,0.001340,1.235732e-04,0.001335,-0.003162,-0.000149,0.000335,0.000771,0.002529,1.0
9,632.933333,23.431387,583.0,613.50,632.0,653.00,676.0,0.851000,0.027959,0.80,...,0.000164,0.001250,3.424638e-04,0.000472,-0.000332,0.000006,0.000185,0.000561,0.001439,1.0


In [263]:
fdfshuf=fdft.sample(frac=1)
fdfshuf=fdft.sample(frac=1)
fdfshuf=fdft.sample(frac=1)

fdftrain=fdfshuf[0:200]
fdftest=fdfshuf[200:]

ftrainx=fdftrain.iloc[0:200,0:965].values
ftrainy=fdftrain.iloc[0:200,966].values

ftestx=fdftest.iloc[0:,0:965].values
ftesty=fdftest.iloc[0:,966].values

scalar=StandardScaler()
fscaledtrainx=scalar.fit_transform(ftrainx)
fscaledtestx=scalar.fit_transform(ftestx)

rdfshuf=rdft.sample(frac=1)
rdfshuf=rdft.sample(frac=1)
rdfshuf=rdft.sample(frac=1)

rdftrain=rdfshuf[0:200]
rdftest=rdfshuf[200:]

rtrainx=rdftrain.iloc[0:200,0:965].values
rtrainy=rdftrain.iloc[0:200,966].values

rtestx=rdftest.iloc[0:,0:965].values
rtesty=rdftest.iloc[0:,966].values

scalar=StandardScaler()
rscaledtrainx=scalar.fit_transform(rtrainx)
rscaledtestx=scalar.fit_transform(rtestx)

In [265]:
base_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=123)
params = [{'C': [0.01, 0.1, 0.5, 1.0, 2.5, 5.0, 10.0, 50.0, 100.0]}]
gs_classifier = GridSearchCV(base_classifier, params, cv=5)
gs_classifier.fit(ftrainx, ftrainy)
print("Best parameter settings F:", gs_classifier.best_params_)
print("Validation accuracy F:%0.6f" % gs_classifier.best_score_)
gs_classifier.fit(rtrainx, rtrainy)
print("Best parameter settings R:", gs_classifier.best_params_)
print("Validation accuracy R: %0.6f" % gs_classifier.best_score_)

Best parameter settings F: {'C': 0.01}
Validation accuracy F:0.680000
Best parameter settings R: {'C': 100.0}
Validation accuracy R: 0.170000


In [268]:
base_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=123)
params = [{'C': [0.01, 0.1, 0.5, 1.0, 2.5, 5.0, 10.0, 50.0, 100.0]}]
gs_classifier = GridSearchCV(base_classifier, params, cv=5)
gs_classifier.fit(fscaledtrainx, ftrainy)
print("Best parameter settings F:", gs_classifier.best_params_)
print("Validation accuracy F: %0.6f" % gs_classifier.best_score_)
gs_classifier.fit(rscaledtrainx, rtrainy)
print("Best parameter settings R:", gs_classifier.best_params_)
print("Validation accuracy R: %0.6f" % gs_classifier.best_score_)

Best parameter settings F: {'C': 0.1}
Validation accuracy F: 0.890000
Best parameter settings R: {'C': 1.0}
Validation accuracy R: 0.650000


In [273]:
from sklearn.svm import SVC
cvals=np.array([0.0001,0.001,0.01,0.1,1.0,5,10.0,100.0,1000.0])
fbesttest=0
fbesttrain=0
fbestc=0
rbesttest=0
rbesttrain=0
rbestc=0
i=0
while i<9:
    classifier = SVC(kernel='rbf', C=cvals[i], random_state=123)
    classifier.fit(fscaledtrainx, ftrainy)
    
    ftrainscore = accuracy_score(ftrainy, classifier.predict(fscaledtrainx))
    ftestscore = accuracy_score(ftesty, classifier.predict(fscaledtestx))
    if ftestscore >= fbesttest:
        fbesttest=ftestscore
        fbesttrain=ftrainscore
        fbestc=cvals[i]
    print("C = "+str(cvals[i]))
    print("Training accuracy: %0.6f" % ftrainscore)
    print("Test accuracy: %0.6f" % ftestscore)
    i=i+1
print('Highest Test Accuracy of '+str(fbesttest)+' with C = '+str(fbestc))
print('Highest Train Accuracy of '+str(fbesttrain)+' with C = '+str(fbestc))
ftesta=classifier.predict(fscaledtestx)
print(ftesta)
print(ftesty)

print("----------ROTATIONS----------")

i=0
while i<9:
    classifier = SVC(kernel='rbf', C=cvals[i], random_state=123)
    classifier.fit(rscaledtrainx, rtrainy)
    
    rtrainscore = accuracy_score(rtrainy, classifier.predict(rscaledtrainx))
    rtestscore = accuracy_score(rtesty, classifier.predict(rscaledtestx))
    if rtestscore >= rbesttest:
        rbesttest=rtestscore
        rbesttrain=rtrainscore
        rbestc=cvals[i]
    print("C = "+str(cvals[i]))
    print("Training accuracy: %0.6f" % rtrainscore)
    print("Test accuracy: %0.6f" % rtestscore)
    i=i+1
print('Highest Test Accuracy of '+str(rbesttest)+' with C = '+str(rbestc))
print('Highest Train Accuracy of '+str(rbesttrain)+' with C = '+str(rbestc))
rtesta=classifier.predict(rscaledtestx)
print(rtesta)
print(rtesty)

C = 0.0001
Training accuracy: 0.215000
Test accuracy: 0.123711
C = 0.001
Training accuracy: 0.215000
Test accuracy: 0.123711
C = 0.01
Training accuracy: 0.215000
Test accuracy: 0.123711
C = 0.1
Training accuracy: 0.215000
Test accuracy: 0.123711
C = 1.0
Training accuracy: 0.990000
Test accuracy: 0.907216
C = 5.0
Training accuracy: 1.000000
Test accuracy: 0.896907
C = 10.0
Training accuracy: 1.000000
Test accuracy: 0.896907
C = 100.0
Training accuracy: 1.000000
Test accuracy: 0.896907
C = 1000.0
Training accuracy: 1.000000
Test accuracy: 0.896907
Highest Test Accuracy of 0.907216494845 with C = 1.0
Highest Train Accuracy of 0.99 with C = 1.0
[ 0.  5.  4.  4.  5.  3.  1.  5.  5.  2.  2.  0.  5.  3.  4.  1.  0.  1.
  1.  4.  5.  2.  2.  5.  2.  3.  3.  0.  1.  1.  0.  4.  5.  1.  3.  4.
  4.  5.  4.  2.  0.  1.  1.  2.  2.  4.  4.  0.  3.  2.  3.  2.  2.  1.
  4.  3.  3.  2.  5.  4.  5.  3.  5.  4.  1.  0.  0.  3.  0.  2.  5.  0.
  3.  4.  0.  5.  2.  0.  0.  4.  3.  3.  5.  0.  4.  2.  0

In [276]:
from sklearn.metrics import confusion_matrix
flabel_names = [0,1,2,3,4,5]
fcm=confusion_matrix(ftesty, ftesta)
fcm

array([[19,  1,  0,  0,  0,  0],
       [ 1, 10,  0,  0,  0,  1],
       [ 0,  0, 16,  0,  0,  0],
       [ 0,  0,  1, 14,  1,  0],
       [ 0,  1,  0,  0, 13,  1],
       [ 0,  0,  0,  0,  3, 15]], dtype=int64)

In [278]:
rlabel_names = [0,1,2,3,4,5,6,7]
rcm=confusion_matrix(rtesty, rtesta)
rcm

array([[ 8,  2,  0,  1,  0,  0,  0,  2],
       [ 0,  9,  2,  0,  0,  0,  0,  1],
       [ 0,  4,  8,  1,  0,  0,  1,  1],
       [ 1,  0,  0,  6,  3,  0,  0,  1],
       [ 0,  0,  0,  0, 12,  1,  0,  0],
       [ 0,  0,  0,  0,  6,  8,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  4,  4],
       [ 0,  0,  0,  1,  0,  0,  1,  9]], dtype=int64)

In [283]:
gvals=np.array( [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7])

fgtrain=np.array([])
fgtest=np.array([])
fgbesttest=0
fgbesttrain=0
fbestg=0

rgtrain=np.array([])
rgtest=np.array([])
rgbesttest=0
rgbesttrain=0
rbestg=0

i=0
while i<6:
    fgrclassifier = SVC(kernel='rbf', gamma=gvals[i], C=fbestc, random_state=123)
    fgrclassifier.fit(fscaledtrainx, ftrainy)
    
    fgrtrainscore = accuracy_score(ftrainy, fgrclassifier.predict(fscaledtrainx))
    fgrtestscore = accuracy_score(ftesty, fgrclassifier.predict(fscaledtestx))
    
    fgtrain=np.append(fgtrain,fgrtrainscore)
    fgtest=np.append(fgtest,fgrtestscore)
    
    if fgrtestscore>fgbesttest:
        fgbesttest=fgrtestscore
        fgbesttrain=fgrtrainscore
        fbestg=gvals[i]
    #print("C = "+str(cvals[i]))
    #print("Training accuracy: %0.6f" % trainscore)
    print('Gamma = '+str(gvals[i]))
    print("Training accuracy: %0.6f" % fgrtrainscore)
    print("Testing accuracy: %0.6f" % fgrtestscore)
    i=i+1

print('Highest Test Accuracy of '+str(fgbesttest)+' with Gamma = '+str(fbestg))
print('Highest Train Accuracy of '+str(fgbesttrain)+' with Gamma = '+str(fbestg))

print("-----------ROTATIONS-----------")

i=0
while i<6:
    rgrclassifier = SVC(kernel='rbf', gamma=gvals[i], C=rbestc, random_state=123)
    rgrclassifier.fit(rscaledtrainx, rtrainy)
    
    rgrtrainscore = accuracy_score(rtrainy, rgrclassifier.predict(rscaledtrainx))
    rgrtestscore = accuracy_score(rtesty, rgrclassifier.predict(rscaledtestx))
    
    rgtrain=np.append(rgtrain,rgrtrainscore)
    rgtest=np.append(rgtest,rgrtestscore)
    
    if rgrtestscore>rgbesttest:
        rgbesttest=rgrtestscore
        rgbesttrain=rgrtrainscore
        rbestg=gvals[i]
    #print("C = "+str(cvals[i]))
    #print("Training accuracy: %0.6f" % trainscore)
    print('Gamma = '+str(gvals[i]))
    print("Training accuracy: %0.6f" % rgrtrainscore)
    print("Testing accuracy: %0.6f" % rgrtestscore)
    i=i+1

print('Highest Test Accuracy of '+str(rgbesttest)+' with Gamma = '+str(rbestg))
print('Highest Train Accuracy of '+str(rgbesttrain)+' with Gamma = '+str(rbestg))

Gamma = 0.01
Training accuracy: 1.000000
Testing accuracy: 0.123711
Gamma = 0.001
Training accuracy: 0.990000
Testing accuracy: 0.907216
Gamma = 0.0001
Training accuracy: 0.675000
Testing accuracy: 0.402062
Gamma = 1e-05
Training accuracy: 0.215000
Testing accuracy: 0.123711
Gamma = 1e-06
Training accuracy: 0.215000
Testing accuracy: 0.123711
Gamma = 1e-07
Training accuracy: 0.215000
Testing accuracy: 0.123711
Highest Test Accuracy of 0.907216494845 with Gamma = 0.001
Highest Train Accuracy of 0.99 with Gamma = 0.001
-----------ROTATIONS-----------
Gamma = 0.01
Training accuracy: 1.000000
Testing accuracy: 0.113402
Gamma = 0.001
Training accuracy: 1.000000
Testing accuracy: 0.659794
Gamma = 0.0001
Training accuracy: 1.000000
Testing accuracy: 0.701031
Gamma = 1e-05
Training accuracy: 1.000000
Testing accuracy: 0.690722
Gamma = 1e-06
Training accuracy: 0.995000
Testing accuracy: 0.680412
Gamma = 1e-07
Training accuracy: 0.430000
Testing accuracy: 0.278351
Highest Test Accuracy of 0.7010

In [288]:
from sklearn.tree import DecisionTreeClassifier
maxd=np.array([3,4,5,6,7,8,9,10])
minl=np.array([1,2,3,4,5])
i=0
num=1000
while i < 8:
    j=0
    while j<5:
        ftclassifier = DecisionTreeClassifier(max_depth=maxd[i], min_samples_leaf=minl[j], random_state=123)
        ftclassifier.fit(fscaledtrainx, ftrainy)
        
        fttrainscore = accuracy_score(ftrainy, ftclassifier.predict(fscaledtrainx))
        fttestscore = accuracy_score(ftesty, ftclassifier.predict(fscaledtestx))
        
        print('Max Depth: '+str(maxd[i])+' |     Min Leaves: '+str(minl[j])+' |     Training Accuracy: '+str(fttrainscore)+' |     Testing Accuracy: '+str(fttestscore))
        #print()
        j=j+1
    i=i+1
    
print("---------ROTATIONS----------")

i=0
num=1000
while i < 8:
    j=0
    while j<5:
        rtclassifier = DecisionTreeClassifier(max_depth=maxd[i], min_samples_leaf=minl[j], random_state=123)
        rtclassifier.fit(rscaledtrainx, rtrainy)
        
        rttrainscore = accuracy_score(rtrainy, rtclassifier.predict(rscaledtrainx))
        rttestscore = accuracy_score(rtesty, rtclassifier.predict(rscaledtestx))
        
        print('Max Depth: '+str(maxd[i])+' |     Min Leaves: '+str(minl[j])+' |     Training Accuracy: '+str(rttrainscore)+' |     Testing Accuracy: '+str(rttestscore))
        #print()
        j=j+1
    i=i+1

Max Depth: 3 |     Min Leaves: 1 |     Training Accuracy: 0.94 |     Testing Accuracy: 0.835051546392
Max Depth: 3 |     Min Leaves: 2 |     Training Accuracy: 0.935 |     Testing Accuracy: 0.835051546392
Max Depth: 3 |     Min Leaves: 3 |     Training Accuracy: 0.93 |     Testing Accuracy: 0.80412371134
Max Depth: 3 |     Min Leaves: 4 |     Training Accuracy: 0.925 |     Testing Accuracy: 0.845360824742
Max Depth: 3 |     Min Leaves: 5 |     Training Accuracy: 0.925 |     Testing Accuracy: 0.845360824742
Max Depth: 4 |     Min Leaves: 1 |     Training Accuracy: 0.98 |     Testing Accuracy: 0.814432989691
Max Depth: 4 |     Min Leaves: 2 |     Training Accuracy: 0.965 |     Testing Accuracy: 0.824742268041
Max Depth: 4 |     Min Leaves: 3 |     Training Accuracy: 0.955 |     Testing Accuracy: 0.80412371134
Max Depth: 4 |     Min Leaves: 4 |     Training Accuracy: 0.945 |     Testing Accuracy: 0.845360824742
Max Depth: 4 |     Min Leaves: 5 |     Training Accuracy: 0.935 |     Testing 

Max Depth: 10 |     Min Leaves: 5 |     Training Accuracy: 0.88 |     Testing Accuracy: 0.639175257732


In [295]:
ftclassifier.feature_importances_

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [289]:
from sklearn.ensemble import RandomForestClassifier
nest=np.array([10,20,30,40,50,60,70,80,90,100])
i=0
while i<10:
    frfclassifier = RandomForestClassifier(n_estimators=nest[i], max_depth=6, min_samples_leaf=2, random_state=123)
    frfclassifier.fit(fscaledtrainx,ftrainy)
    
    frftrainscore = accuracy_score(ftrainy, frfclassifier.predict(fscaledtrainx))
    frftestscore = accuracy_score(ftesty, frfclassifier.predict(fscaledtestx))
    
    print('Training Accuracy: '+str(frftrainscore)+' |     Testing Accuracy: '+str(frftestscore))
    i=i+1
    
print()
print("----------ROTATIONS-----------")
print()
i=0
while i<10:
    rrfclassifier = RandomForestClassifier(n_estimators=nest[i], max_depth=6, min_samples_leaf=2, random_state=123)
    rrfclassifier.fit(rscaledtrainx,rtrainy)
    
    rrftrainscore = accuracy_score(rtrainy, rrfclassifier.predict(rscaledtrainx))
    rrftestscore = accuracy_score(rtesty, rrfclassifier.predict(rscaledtestx))
    
    print('Training Accuracy: '+str(rrftrainscore)+' |     Testing Accuracy: '+str(rrftestscore))
    i=i+1

Training Accuracy: 0.97 |     Testing Accuracy: 0.907216494845
Training Accuracy: 0.99 |     Testing Accuracy: 0.938144329897
Training Accuracy: 0.99 |     Testing Accuracy: 0.927835051546
Training Accuracy: 0.995 |     Testing Accuracy: 0.927835051546
Training Accuracy: 0.99 |     Testing Accuracy: 0.917525773196
Training Accuracy: 0.995 |     Testing Accuracy: 0.927835051546
Training Accuracy: 0.99 |     Testing Accuracy: 0.927835051546
Training Accuracy: 0.99 |     Testing Accuracy: 0.927835051546
Training Accuracy: 0.995 |     Testing Accuracy: 0.938144329897
Training Accuracy: 0.995 |     Testing Accuracy: 0.948453608247

----------ROTATIONS-----------

Training Accuracy: 0.975 |     Testing Accuracy: 0.577319587629
Training Accuracy: 1.0 |     Testing Accuracy: 0.659793814433
Training Accuracy: 1.0 |     Testing Accuracy: 0.731958762887
Training Accuracy: 1.0 |     Testing Accuracy: 0.773195876289
Training Accuracy: 1.0 |     Testing Accuracy: 0.701030927835
Training Accuracy: 1.

In [291]:
from sklearn.neural_network import MLPClassifier
layer1=np.array([50,100,200,300,400,500])
layer2=np.array([20,50,100,200,300,400,500])
i=0
while i<6:
    j=0
    while j<7:
        fmlpclassifier = MLPClassifier(hidden_layer_sizes=(layer1[i],layer2[j]), random_state=123)
        fmlpclassifier.fit(fscaledtrainx,ftrainy)

        fmlptrainscore = accuracy_score(ftrainy, fmlpclassifier.predict(fscaledtrainx))
        fmlptestscore = accuracy_score(ftesty, fmlpclassifier.predict(fscaledtestx))
        print('Layer 1: '+str(layer1[i])+'     Layer 2: '+str(layer2[j]))
        print('Training Accuracy: '+str(fmlptrainscore))
        print('Testing Accuracy: '+str(fmlptestscore))
        j=j+1
    i=i+1   

print()
print("--------ROTATIONS--------")
print()
i=0
while i<6:
    j=0
    while j<7:
        rmlpclassifier = MLPClassifier(hidden_layer_sizes=(layer1[i],layer2[j]), random_state=123)
        rmlpclassifier.fit(rscaledtrainx,rtrainy)

        rmlptrainscore = accuracy_score(rtrainy, rmlpclassifier.predict(rscaledtrainx))
        rmlptestscore = accuracy_score(rtesty, rmlpclassifier.predict(rscaledtestx))
        print('Layer 1: '+str(layer1[i])+'     Layer 2: '+str(layer2[j]))
        print('Training Accuracy: '+str(rmlptrainscore))
        print('Testing Accuracy: '+str(rmlptestscore))
        j=j+1
    i=i+1    
# code for 3.1 here

Layer 1: 50     Layer 2: 20
Training Accuracy: 1.0
Testing Accuracy: 0.855670103093
Layer 1: 50     Layer 2: 50
Training Accuracy: 1.0
Testing Accuracy: 0.855670103093
Layer 1: 50     Layer 2: 100
Training Accuracy: 1.0
Testing Accuracy: 0.865979381443
Layer 1: 50     Layer 2: 200
Training Accuracy: 1.0
Testing Accuracy: 0.824742268041
Layer 1: 50     Layer 2: 300
Training Accuracy: 1.0
Testing Accuracy: 0.855670103093
Layer 1: 50     Layer 2: 400
Training Accuracy: 1.0
Testing Accuracy: 0.835051546392
Layer 1: 50     Layer 2: 500
Training Accuracy: 1.0
Testing Accuracy: 0.865979381443
Layer 1: 100     Layer 2: 20
Training Accuracy: 1.0
Testing Accuracy: 0.855670103093
Layer 1: 100     Layer 2: 50
Training Accuracy: 1.0
Testing Accuracy: 0.917525773196
Layer 1: 100     Layer 2: 100
Training Accuracy: 1.0
Testing Accuracy: 0.835051546392
Layer 1: 100     Layer 2: 200
Training Accuracy: 1.0
Testing Accuracy: 0.865979381443
Layer 1: 100     Layer 2: 300
Training Accuracy: 1.0
Testing Accu

In [292]:
fcclassifier = MLPClassifier(hidden_layer_sizes=(50,100), random_state=123)
fcclassifier.fit(fscaledtrainx,ftrainy)

fcctrainscore = accuracy_score(ftrainy, fmlpclassifier.predict(fscaledtrainx))
fctestscore = accuracy_score(ftesty, fcclassifier.predict(fscaledtestx))
print('Training Accuracy: '+str(fcctrainscore))
print('Testing Accuracy: '+str(fctestscore))

print("----------ROTATIONS----------")

rcclassifier = MLPClassifier(hidden_layer_sizes=(50,100), random_state=123)
rcclassifier.fit(rscaledtrainx,rtrainy)

rcctrainscore = accuracy_score(rtrainy, rmlpclassifier.predict(rscaledtrainx))
rctestscore = accuracy_score(rtesty, rcclassifier.predict(rscaledtestx))
print('Training Accuracy: '+str(rcctrainscore))
print('Testing Accuracy: '+str(rctestscore))

Training Accuracy: 1.0
Testing Accuracy: 0.865979381443
----------ROTATIONS----------
Training Accuracy: 1.0
Testing Accuracy: 0.587628865979
